In [1]:
import pandas as pd
import torch
import yaml
from torch import cosine_similarity

from model.encoder import Encoder
from model.recommender import DeepFM
from utils.data import get_feature_sizes

/home/archy1/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
This can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64

2024-10-04 16:16:01.310684: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-04 16:16:01.312925: I external

In [2]:
movies = pd.read_csv("data/ml-20m/movies.csv", header=0, names=["movie_id", "movie_title", "genres"])

movies = movies[["movie_id", "movie_title"]]

ratings = pd.read_csv("data/ml-20m/ratings.csv", header=0, names=["user_id", "movie_id", "rating", "timestamp"])
ratings = ratings[["user_id", "movie_id", "rating"]]

requests = pd.read_csv('data/ml-20m/requests.csv')
requests = requests.groupby("movie_id").agg({
    "movie_title": "first",
    "request": list,
}).reset_index()
requests.set_index("movie_id", inplace=True, drop=False)

user_id_to_unique_id = {user_id: i for i, user_id in enumerate(ratings["user_id"].unique())}
item_id_to_unique_id = {movie_id: i for i, movie_id in enumerate(requests["movie_id"].unique())}

In [3]:
args = yaml.safe_load(open("configs/collaborative.yaml", "r"))

device = "cuda" if torch.cuda.is_available() else "cpu"

recommender = DeepFM(feature_dims=get_feature_sizes(ratings), **args["recommender"]).to(device)
recommender.load_state_dict(torch.load("weights/collaborative/deepfm.pt", map_location=device))

movie_embeddings = recommender.embedding.embedding.weight[recommender.embedding.offsets[1]:].cpu()

encoder = Encoder().to(device)
encoder.load_state_dict(torch.load("weights/collaborative/encoder.pt", map_location=device))

/home/archy1/mambaforge/envs/crecs/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


<All keys matched successfully>

In [4]:
user_id = 36
unique_movie_ids = torch.tensor([item_id_to_unique_id[movie_id] for movie_id in requests["movie_id"].unique()])
uniquer_user_id = torch.full_like(unique_movie_ids, user_id_to_unique_id[user_id])

In [8]:
ratings[ratings["user_id"] == user_id].merge(movies, on="movie_id").sort_values("rating", ascending=False)

,user_id,movie_id,rating,movie_title
16,36,2353,4.0,Enemy of the State (1998)
0,36,145,3.5,Bad Boys (1995)
8,36,1358,3.5,Sling Blade (1996)
13,36,2023,3.5,"Godfather: Part III, The (1990)"
9,36,1408,3.5,"Last of the Mohicans, The (1992)"
10,36,1597,3.5,Conspiracy Theory (1997)
19,36,58293,3.5,"10,000 BC (2008)"
6,36,1088,3.0,Dirty Dancing (1987)
1,36,163,3.0,Desperado (1995)
15,36,2302,3.0,My Cousin Vinny (1992)


In [9]:
request = "I want to watch a mob movie"

request_embedding = encoder(request).cpu()

similarities = cosine_similarity(request_embedding, movie_embeddings)

_, indices = torch.topk(similarities, k=20, largest=True)

movies.iloc[indices]

,movie_id,movie_title
954,1089,Reservoir Dogs (1992)
3487,4011,Snatch (2000)
1696,2000,Lethal Weapon (1987)
1047,1213,Goodfellas (1990)
2208,2542,"Lock, Stock & Two Smoking Barrels (1998)"
5,6,Heat (1995)
128,145,Bad Boys (1995)
1649,1953,"French Connection, The (1971)"
4902,5782,"Professional, The (Le professionnel) (1981)"
1697,2001,Lethal Weapon 2 (1989)


In [11]:
rec_features = torch.stack([uniquer_user_id, unique_movie_ids], dim=-1).to(device)

user_ratings = recommender(rec_features).cpu()

rankings = similarities * user_ratings

_, indices = torch.topk(rankings, k=20, largest=True)

movies.iloc[indices]

,movie_id,movie_title
954,1089,Reservoir Dogs (1992)
3487,4011,Snatch (2000)
1047,1213,Goodfellas (1990)
2208,2542,"Lock, Stock & Two Smoking Barrels (1998)"
1696,2000,Lethal Weapon (1987)
1649,1953,"French Connection, The (1971)"
4902,5782,"Professional, The (Le professionnel) (1981)"
2853,3275,"Boondock Saints, The (2000)"
128,145,Bad Boys (1995)
5,6,Heat (1995)
